In [123]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from matplotlib.pyplot import figure
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, recall_score, precision_score

In [124]:
def calculate_score(data_frame, score_type, no_sensors):
    """
    :param data_frame: Complete Dataframe
    :param no_sensors: Number of Sensors in Dataframe
    :param start_time: Start point. Value from time column
    :param max_delta: Number of samples. max_time = (no_samples * (time between two samples))
    :param step_size: It is the number of samples added to current time_delta to get next time_delta
    :return:
    """
    labels_pred = data_frame.iloc[:, (3 * no_sensors) + 1:(4 * no_sensors) + 1]
    labels_true = data_frame.iloc[:, (4 * no_sensors) + 1].values
    
    result_score = np.zeros((no_sensors))

    for sensor in range(no_sensors):
        anomaly_pred = labels_pred.iloc[:, sensor].values
        if score_type == "f1_score":
            result = f1_score(labels_true, anomaly_pred, average="binary")
            index_names = ["f1 sensor_" + str(i) for i in range(no_sensors)]
        elif score_type == "precision_score":
            result = precision_score(labels_true, anomaly_pred, average="binary")
            index_names = ["precision sensor_" + str(i) for i in range(no_sensors)]
        elif score_type == "recall_score":
            result = recall_score(labels_true, anomaly_pred, average="binary")
            index_names = ["recall sensor_" + str(i) for i in range(no_sensors)]   
        else:
            result = 0
            index_names = ["error" for i in range(no_sensors)]
            
        result_score[sensor] = result

    score_df = pd.Series(data=result_score, index=index_names)
    return score_df

In [125]:
def get_confusion_matrix(data_frame, no_sensors, specific_sensor):
    labels_pred = data_frame.iloc[:, (3 * no_sensors) + 1:(4 * no_sensors) + 1]
    labels_true = data_frame.iloc[:, (4 * no_sensors) + 1].values
    
    # No errors are on sawtooth signal (labels_true in dataframe is regarding sine wave)
    if specific_sensor == 1:
        labels_true = np.full((data_frame.shape[0]), 0)
    
    anomaly_pred = labels_pred.iloc[:, specific_sensor].values
    return confusion_matrix(labels_true, anomaly_pred).ravel()

In [126]:
def visualise_metric_per_sensor(results, title):
    fig, axes = plt.subplots(results.shape[1]-1, 1, figsize=(10,20),constrained_layout=False)
    ax = axes.ravel()
    t = results.loc[:,"delta_t"]
    columns = results.columns
    for i in range(results.shape[1]-1): 
        sns.lineplot(data=results, 
                     x=t, 
                     y=columns[i], 
                     ax=ax[i],
                     linewidth=1,
                     color="black")
        ax[i].set_xlabel("delta t [in samples]")
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.suptitle(title, fontsize=16)

In [127]:
def visualise_metric_machine(results, score_type, phase):
    t = results.loc[:,"delta_t"]
    complete_title = "CPPS Data - Beginning of Phase '{}''".format(phase)
    
    # Caluculate Metric for hole machine (sum over sensors and devide by no_sensors)
    labels = results.drop(columns="delta_t", axis=0)
    result_machine = labels.sum(axis=1) / results.shape[1]
    
    # Visualise Results
    sns.lineplot(x=t, 
                 y=result_machine, 
                 linewidth=1,
                 color="black")
    plt.xlabel("delta t [in samples]")
    plt.ylabel("{} over all dim".format(score_type))
    #plt.tight_layout()
    plt.title(complete_title, fontsize=16, y=1.12)

# Evaluation of Euclidian Distance Metric
## Setup

In [128]:
all_data = pd.read_csv("../../files/classification/MSE/artfic_max_minus_60percent.csv", sep=";")

In [129]:
all_data.head()

,ID,sine_signal target,sawtooth_signal target,sine_signal predicted,sawtooth_signal predicted,sine_signal reconstruction error,sawtooth_signal reconstruction error,Anomaly Sensor_1,Anomaly Sensor_2,anomaly
0,8.0,1.253286,-0.986644,1.237256,-0.277459,0.016029,0.709186,0,0,0
1,9.0,1.313607,-0.618104,1.304106,-0.529158,0.009501,0.088946,0,0,0
2,10.0,1.429184,-0.357066,1.316485,-0.459979,0.112699,0.102914,0,0,0
3,11.0,1.376550,0.055021,1.388720,-0.267551,0.012169,0.322571,0,0,0
4,12.0,1.164972,0.153692,1.383462,0.145293,0.218489,0.008399,0,0,0


# Evaluation Metrics
## F1-score

In [130]:
f1_score = calculate_score(all_data, "f1_score", 2)
print("F1 score: {}".format(f1_score[0]))

F1 score: 0.12703583061889248


## Precision Score

In [131]:
precision_score = calculate_score(all_data, "precision_score", 2)
print("Precision score: {}".format(precision_score[0]))

Precision score: 0.09420289855072464


In [118]:
print(precision_score)

precision sensor_0    0.094203
precision sensor_1    0.047856
dtype: float64


## Recall Score

In [119]:
recall_score = calculate_score(all_data, "recall_score", 2)
print("Recall score: {}".format(recall_score[0]))

Recall score: 0.195


In [120]:
print(recall_score)

recall sensor_0    0.195
recall sensor_1    0.080
dtype: float64


## Confusion Matrix
### Sensor with errors

In [121]:
print("Positive --> Anomaly")
print("Negative --> Normal Behaviour")
print("--"*15)
tn, fp, fn, tp = get_confusion_matrix(all_data, 2, 0)
print("Sensor No. {}:".format(1))
print("True negative: {}".format(tn))
print("False positive: {}".format(fp))
print("False negative: {}".format(fn))
print("True positive: {}".format(tp))

Positive --> Anomaly
Negative --> Normal Behaviour
------------------------------
Sensor No. 1:
True negative: 10225
False positive: 1125
False negative: 483
True positive: 117


### Sensor without errors

In [122]:
print("Positive --> Anomaly")
print("Negative --> Normal Behaviour")
print("--"*15)
tn, fp, fn, tp = get_confusion_matrix(all_data, 2, 1)
print("Sensor No. {}:".format(2))
print("True negative: {}".format(tn))
print("False positive: {}".format(fp))
print("False negative: {}".format(fn))
print("True positive: {}".format(tp))

Positive --> Anomaly
Negative --> Normal Behaviour
------------------------------
Sensor No. 2:
True negative: 10947
False positive: 1003
False negative: 0
True positive: 0
